In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers as kl
from tensorflow.keras import models as km
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks as kc
from tensorflow.keras import optimizers as ko
from tensorflow.keras import regularizers as kr

In [ ]:
# Test if GPU is available for TensorFlow
print(f"GPU : {tf.config.list_physical_devices('GPU')}")
# Check TensorFlow version
print(f"TF version : {tf.__version__}")

In [ ]:
DDIR = '/home/masterdesky/data/CAMELS/2D_maps/data/'
FILE = os.listdir(DDIR)
FILE = sorted([
    f for f in FILE if ('.txt' not in f) & ('_CV_' not in f)
])

In [ ]:
FILE

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Load a selected datafile
f = FILE[5]
X = np.load(os.path.join(DDIR, f), allow_pickle=True)
X = np.log10(X)

# Load labels
dataset = '_'.join(f.split('_')[2:-2])
y = np.genfromtxt(os.path.join(DDIR, f"params_{dataset}.txt"))
y = np.repeat(y, 15, axis=0)

test_size = 0.33
valid_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y,
                          test_size=test_size, random_state=57)
del(X)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train,
                          test_size=valid_size/(1-test_size), random_state=57)

print('Train :', X_train.shape)
print('Valid :', X_valid.shape)
print('Test :', X_test.shape)

#
# TENSORFLOW PURGATORY IN EARLY 2022
#
# Wrap data in Dataset objects
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
del(X_train); del(y_train)
del(X_valid); del(y_valid)

# The batch size must now be set on the Dataset objects
batch_size = 128
train_data = train_data.batch(batch_size)
valid_data = valid_data.batch(batch_size)

# Disable AUTO sharding policy
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = \
                        tf.data.experimental.AutoShardPolicy.OFF
train_data = train_data.with_options(options)
valid_data = valid_data.with_options(options)

In [ ]:
class MultiCNN:
    '''
    Creates a CNN model for regression and classification problems with
    multiple types of variables as their outputs. The model supports
    square shaped images.
    
    Parameters
    ----------
    imsize : 
    '''
    def __init__(self,
                 imsize, n_channels,
                 num_filters, kernelsize, padding, stride, kreg,
                 activation):
        self.imsize = imsize
        self.n_channels = n_channels
        self.num_filters = num_filters
        self.kernelsize = kernelsize
        self.padding = padding
        self.stride = stride
        self.kreg = kreg
        self.activation = activation

        self.inputs = kl.Input(shape=(imsize, imsize, n_channels))
        self.branches = []
        
    def __cnn__(self, inputs):
        #
        # Convolutional block 1.
        # 3x3CONVx32 -> 3x3CONVx32 -> MAXPOOLx2
        #
        x = kl.Conv2D(filters=self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(inputs)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))
        
        x = kl.Conv2D(filters=self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))
        
        x = kl.MaxPooling2D(strides=(2, 2))(x)


        #
        # Convolutional block 2.
        # 3x3CONVx64 -> 3x3CONVx64 -> MAXPOOLx2
        #
        x = kl.Conv2D(filters=2*self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.Conv2D(filters=2*self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))
        
        x = kl.MaxPooling2D(strides=(2, 2))(x)


        #
        # Convolutional block 3.
        # 3x3CONVx128 -> 1x1CONVx64 -> 3x3CONVx128 -> MAXPOOLx2
        #
        x = kl.Conv2D(filters=4*self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.Conv2D(filters=2*self.num_filters,
                    kernel_size=(1, 1),
                    padding=self.padding,
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.Conv2D(filters=4*self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.MaxPooling2D(strides=(2, 2))(x)


        #
        # Convolutional block 4.
        # 3x3CONVx256 -> 1x1CONVx128 -> 3x3CONVx256 -> MAXPOOLx2
        #
        x = kl.Conv2D(filters=8*self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.Conv2D(filters=4*self.num_filters,
                    kernel_size=(1, 1),
                    padding=self.padding,
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.Conv2D(filters=8*self.num_filters,
                    kernel_size=(self.kernelsize, self.kernelsize),
                    padding=self.padding,
                    strides=(self.stride, self.stride),
                    kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.MaxPooling2D(strides=(2, 2))(x)


        #
        # Convolutional block 5.
        # 3x3CONVx512 -> 1x1CONVx256 -> 3x3CONVx512 -> AVGPOOL
        #
        x = kl.Conv2D(filters=16*self.num_filters,
                      kernel_size=(self.kernelsize, self.kernelsize),
                      padding=self.padding,
                      strides=(self.stride, self.stride),
                      kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.Conv2D(filters=8*self.num_filters,
                      kernel_size=(1, 1),
                      padding=self.padding,
                      kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))

        x = kl.Conv2D(filters=16*self.num_filters,
                      kernel_size=(self.kernelsize, self.kernelsize),
                      padding=self.padding,
                      strides=(self.stride, self.stride),
                      kernel_regularizer=kr.l2(self.kreg))(x)
        x = kl.Activation(self.activation)(kl.BatchNormalization()(x))
        
        return x
        
    def add_branch(self,
                   n_target, branch_name):
        '''
        Parameters
        ----------
        n_target : int
            Number of target values on this branch. Set `n_target = 1`
            for regression problems, while `n_targets > 1` for
            classification.
        
        branch_name : str
            Arbitrary name given to the branch.
        '''
        assert branch_name is not None, "Every branch should have a name!"
        ## CNN
        x = self.__cnn__(self.inputs)
        # Branch-specific parts
        x = kl.GlobalAveragePooling2D()(x)
        if n_target == 1: activation = None
        else: activation = 'softmax'
        x = kl.Dense(units=n_target, activation=activation,
                     name = f"{branch_name}")(x)
        
        self.branches.append(x);

    def get_model(self):
        assert len(self.branches) > 0, "No target branches added yet!"
        return km.Model(inputs=self.inputs, outputs=self.branches,
                        name="multicnn_net")

In [ ]:
gpu = '0'
GPU = [f"GPU:{i}" for i in gpu.split(',')]

if len(gpu.split(',')) > 1:
    strategy = tf.distribute.MirroredStrategy(GPU)
else:
    strategy = tf.distribute.OneDeviceStrategy(GPU[0])

with strategy.scope():
    # Initialize the CNN template for all network branches
    multi_cnn = MultiCNN(
        imsize=256,
        n_channels=1,
        num_filters=32,
        kernelsize=3,
        padding='same',
        stride=1,
        kreg=5e-05,
        activation='relu'
    )
    
    # Add branches for all target value
    multi_cnn.add_branch(n_target=1, branch_name="Omega_m")
    multi_cnn.add_branch(n_target=1, branch_name="sigma_8")
    multi_cnn.add_branch(n_target=1, branch_name="A_SN1")
    multi_cnn.add_branch(n_target=1, branch_name="A_AGN1")
    multi_cnn.add_branch(n_target=1, branch_name="A_SN2")
    multi_cnn.add_branch(n_target=1, branch_name="A_AGN2")
    
    # Compile the model 
    model = multi_cnn.get_model()
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))
    
    # Create callback checkpoint
    best_model = kc.ModelCheckpoint('best_model.hdf5',
                                save_best_only=True, verbose=1)

In [ ]:
model.summary()

In [ ]:
# Fit the model
epochs = 5
batch_size = 128
history = model.fit(train_data,#x=X_train, y=y_train,
                    validation_data=valid_data,#(X_valid, y_valid),
                    epochs=epochs,#batch_size=batch_size,
                    callbacks=[best_model])